algoritmo que: 
* a gestão de filas de espera em hospitais envolve a classificação de risco e priorização dos pacientes;
* monitorar a quantidade de pessoas inscritas e o tempo de espera;
* definir a prioridade de atendimento. Cada cor tem um tempo de espera tolerável, que indica a gravidade do caso:
    * Vermelho: risco de morte, atendimento imediato;
     * Laranja: tempo de espera tolerável de até 10 minutos;
    * Amarelo: tempo de espera tolerável de até 20 minutos;
    * Verde: não há risco de morte, tempo de espera tolerável de até 60 minutos;
    * Azul: quadros crônicos, encaminhamento ao centro de saúde, tempo de espera tolerável de até 180 minutos.
* organize a agenda de pacientes, devendo considerar:
    * o número de consultas marcadas;
    * o tempo médio de duração e a previsão de contratempos.

Em observação à lei abaixo, esta será considerada na implementação do algoritmo:
* A Lei 10.048/00 garante atendimento prioritário a idosos com 60 anos ou mais na administração pública.
* Também inclui pessoas do seguinte perfil: deficientes, gestantes, lactantes, pessoas com crianças de colo e obesos.
* Todavia, indivíduos com idade 80+ tem preferência sob todos os demais.

In [ ]:
# !pip install faker
from faker import Faker
import random

import time

import pandas as pd
import datetime

### Gerando dados para teste

In [ ]:
# Inicializando o Faker e configurando para nomes em português
fake = Faker("pt_BR")

# Inicializando as listas
lista_nomes = []          # string: primeiro nome ou nome completo/nome social
lista_dt_nascimento = []  # date: a data de nascimento
lista_prioridade = []     # bool: False para NÃO; True para SIM
lista_dt_entrada = []     # datetime: indica o horário de cadastro
lista_especialidade = []  # string: especialidade médica (geral, neurologia, oftalmologista etc)
lista_tma = []            # int: minutos com o tempo médio de atendimento para cada especialidade
lista_class_risco = []    # int: [0:1] representando azul, verde, laranja, amarelo, vermelho

lista_colunas = ['Nome', 'Nascimento', 'Prioridade', 'Entrada', 'Especialidade', 'TMA', 'Risco']
df = pd.DataFrame(columns=lista_colunas)

# Opções para os campos
especialidades_exemplo = ["geral", "neurologia", "oftalmologia", "cardiologia", "dermatologia", "pediatria"]
#class_risco_exemplo = ["vermelho", "laranja", "amarelo", "verde", "azul"]
class_risco_exemplo = [4, 3, 2, 1, 0]
# Função para gerar uma data de nascimento aleatória
def gerar_data_nascimento():
    start_date = datetime.date(1940, 1, 1)
    end_date = datetime.date(2005, 12, 31)
    return start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))


# Função para gerar uma data de entrada aleatória com horário
def gerar_data_entrada(is_random=True):
    now = datetime.datetime.now()
    if is_random:        
        delta_days = random.randint(0, 0)
        delta_seconds = random.randint(0, 86400)  # Número de segundos no dia
        return now - datetime.timedelta(days=delta_days, seconds=delta_seconds)
    else: return now

    
# Populando as listas com 100 registros
for _ in range(100):
    time.sleep(0.05)
    lista_nomes.append(fake.first_name())  # Gera um primeiro nome aleatório em português
    lista_dt_nascimento.append(gerar_data_nascimento())
    lista_prioridade.append(random.choice([True, False]))
    lista_dt_entrada.append(gerar_data_entrada(False))
    lista_especialidade.append(random.choice(especialidades_exemplo))
    lista_tma.append(random.choice([15, 30, 45, 60]))  # Tempo médio de atendimento entre 10 e 60 minutos
    lista_class_risco.append(random.choice(class_risco_exemplo))
    
lista_dados = [lista_nomes, lista_dt_nascimento, lista_prioridade, lista_dt_entrada,
               lista_especialidade, lista_tma, lista_class_risco]
    
for i, coluna in enumerate(lista_colunas):
    df[coluna] = lista_dados[i]

df

### Iniciando funções e classes

In [ ]:
class Fila():
    """
    Classe que representa uma fila de usuários, com funcionalidades para inserir e 
    remover usuários, verificar o topo e o tempo total de espera.

    A fila armazena usuários e calcula o tempo total médio de atendimento (TMA) com base 
    nos usuários presentes. Métodos incluem a inserção e remoção de usuários em posições 
    específicas, exibição dos dados e cálculo do tempo de espera total.

    Atributos:
    - total_tma (int): Armazena o tempo total de espera acumulado dos usuários na fila.
    - data (list): Lista de usuários na fila.

    Métodos:
    - inserir(usuario, indice=''): Insere um usuário na posição especificada ou no final.
    - remover(indice=''): Remove e retorna o usuário da posição especificada ou do início.
    - topo(): Retorna o usuário no topo da fila, se houver.
    - vazio(): Verifica se a fila está vazia.
    - quantidade(): Retorna a quantidade de usuários na fila.
    - exibir(): Retorna uma lista com todos os usuários na fila.
    - tempo_espera(): Retorna o tempo total de espera acumulado na fila.
    """



    def __init__(self):        
        self.data = []
        self.total_tma = 0
        
        

    def inserir(self, usuario, indice=''):
        """
        Insere um usuário na fila, atualizando o total de tempo de espera (TMA).
        
        Parâmetros:
        - usuario (dict | pd.Series): O usuário a ser inserido, contendo o campo 'TMA'.
        - indice (int, opcional): A posição na qual o usuário deve ser inserido. 
          Se não especificado, o usuário é adicionado ao final.

        Retorna:
        - None.
        """
        
        if indice == '': self.data.append(usuario)
        else: self.data.insert(indice, usuario)
        self.total_tma+= usuario['TMA']

        
        
    def remover(self, indice=''):
        """
        Remove e retorna um usuário da fila, atualizando o total de tempo de espera (TMA).
        
        Parâmetros:
        - indice (int, opcional): A posição do usuário a ser removido. Se não especificado, 
          o usuário é removido do início da fila.

        Retorna:
        - dict: O usuário removido.
        """
        
        if len(self.data) > 0:
            if type(indice) is not int: usuario = self.data.pop(0)
            else: usuario = self.data.pop(indice)
            
            self.total_tma-= usuario['TMA']
            return usuario

        

    def topo(self):
        """
        Retorna o usuário no topo da fila, se houver.
        
        Retorna:
        - dict: O primeiro usuário na fila ou None se a fila estiver vazia.
        """
        
        if len(self.data) > 0:
            return self.data[0]



    def vazio(self):
        """
        Verifica se a fila está vazia.
        
        Retorna:
        - bool: True se a fila estiver vazia, False caso contrário.
        """
        
        return not len(self.data) > 0



    def quantidade(self):
        """
        Retorna a quantidade de usuários na fila.
        
        Retorna:
        - int: O número de usuários na fila.
        """

        return len(self.data)



    def exibir(self):
        """
        Retorna uma lista com todos os usuários na fila.
        
        Retorna:
        - list: Lista contendo todos os usuários na fila.
        """

        return list(self.data)



    def tempo_espera(self):
        """
        Retorna o tempo total de espera acumulado na fila.
        
        Retorna:
        - int: O tempo total de espera (TMA) dos usuários na fila.
        """

        return self.total_tma
        
        
    
    
def calc_idade(nascido):
    """
    Calcula a idade de um cliente com base na data de nascimento fornecida.

    A função calcula a idade atual do cliente subtraindo o ano de nascimento do
    ano atual. Ajusta a idade caso o mês e o dia de nascimento ainda não tenham
    ocorrido no ano atual.

    Parâmetros:
    - nascido (datetime.date): A data de nascimento do cliente.

    Retorna:
    - int: A idade do usuário em anos.
    """
    
    now = datetime.datetime.now()
    return now.year - nascido.year - ((now.month, now.day) < (nascido.month, nascido.day))



def inserir_usuario(usuario):
    """
    Insere um usuário em uma fila com base em sua idade e prioridade, 
    adicionando idade e senha ao seu registro e classificando-o conforme critérios de risco.

    A função calcula a idade do usuário, gera uma senha específica e o coloca na fila 
    apropriada: 'Prioridade Extra', 'Prioridade 1' ou 'Normal'. A senha é formada a 
    partir dos dados de especialidade do usuário e outros critérios, com um prefixo 
    adicional para usuários prioritários.

    Parâmetros:
    - usuario (dict ou pandas.Series): Objeto contendo as informações do usuário, incluindo 
      'Nascimento', 'Especialidade', 'Nome' e um campo opcional 'Prioridade'.

    Retorna:
    - None. Exibe uma mensagem indicando a fila de inserção e a posição do usuário na fila.
    """

    idade = calc_idade(usuario['Nascimento'])
    usuario['Idade'] = idade    
    senha = str(usuario['Especialidade'][:5]+'-'+str(row[0])).upper()    
    
    if idade >= 80:
        senha = 'PE-'+senha    # obs.:PE > Prioridade Extra
        nome_fila = 'Prioridade 2'
        pos = manejar_risco(fila_priori2, usuario)
        
    elif idade >= 60 or usuario['Prioridade']:
        senha = 'P-'+senha
        nome_fila = 'Prioridade 1'
        pos = manejar_risco(fila_priori1, usuario)
        
    else:
        nome_fila = 'Normal'
        pos = manejar_risco(fila_normal, usuario)
        
    usuario['Senha'] = senha

    print('Usuário {}, senha: {}. Foi inserido na posição {} da fila {}'
          .format(usuario['Nome'], usuario['Senha'], pos, nome_fila))
    
    
    
def manejar_risco(fila, usuario_novo):
    """
    Define a posição de inserção do novo usuário em uma fila com base em seu grau de risco.

    A função compara o grau de risco do novo usuário com os usuários já na fila, 
    posicionando-o de forma a manter a organização por risco crescente. Se o risco do 
    novo usuário não é válido (menor que 5 ou não inteiro), exibe uma mensagem de erro.

    Parâmetros:
    - fila (objeto Fila): A fila onde o usuário será inserido.
    - usuario_novo (dict ou pandas.Series): Objeto com as informações do novo usuário, 
      incluindo o campo 'Risco'.

    Retorna:
    - int: A posição na qual o novo usuário foi inserido na fila. Exibe mensagem de erro 
      e não insere o usuário se o valor de risco for inválido.
    """
    
    risco_usuario_novo = usuario_novo['Risco']
    pos = 0
    if risco_usuario_novo < 5 or type(risco_usuario_novo) is not int:
        if not fila.vazio():
            for i in range(fila.quantidade() - 1, -1, -1):
                usuario_fila = fila.exibir()[i]
                if risco_usuario_novo <= usuario_fila.Risco:
                    pos = i + 1
                    fila.inserir(usuario_novo, pos)
                    break
                elif risco_usuario_novo > usuario_fila.Risco and i != 0:
                    continue
                elif i == 0:
                    pos = i
                    fila.inserir(usuario_novo, pos)            
        else: fila.inserir(usuario_novo)
        return pos
    else: print('Erro')

Há duas, principais, alternativas: 
1. Trabalhar com uma única fila, e dentro desta operar inserções e remanejos conforme necessário para adequar aos três graus de prioridades.
2. Fazer uso de três filas, uma para cada prioridade, mais intuitivo e fácil de implementar.

Neste primeiro momento, acabarei optando pela segunda alternativa.

In [ ]:
# iniciando cada objeto fila
fila_normal = Fila()
fila_priori1 = Fila()
fila_priori2 = Fila()

In [ ]:
# simulando a inserção de vários registros, em cada fila baseado na PREFERENCIA
for row in df.iterrows():
    user = row[1].copy()
    inserir_usuario(user)

### Quantidade em cada fila

In [ ]:
fila_normal.quantidade()

In [ ]:
fila_priori1.quantidade()

In [ ]:
fila_priori2.quantidade()

In [ ]:
# mostrar dados da fila
fila_normal.exibir()

### Inserção de único usuário

In [ ]:
# simulando as inserções na fila, com ajustes baseado no GRAU DE RISCO
# aviso: em algum momento, as inserções DEVEM respeitar alguma medida de tempo,
# caso contrário riscos de grau risco 4 (azul), serem atendidos muito tardiamente ou não serem atendidos

dict_user = {'Nome': 'Avril Lavigne',
             'Nascimento': datetime.date(1984, 9, 27),
             'Prioridade': False,
             'Entrada': gerar_data_entrada(False),
             'Especialidade': 'dermatologia',
             'TMA': 30,
             'Risco': 3}

inserir_usuario(dict_user)

In [ ]:
user = fila_normal.exibir()[17]
user['Nome'], user['Idade'], user['Prioridade']

In [ ]:
fila_normal.exibir()[17]['Nascimento']

In [ ]:
# remoção de um usuário à fila
_ = fila_normal.remover(10)
_

In [ ]:
# tempo de espera em minutos
fila_normal.tempo_espera()

In [ ]:
fila_priori2.tempo_espera()

In [ ]:
fila_priori1.tempo_espera()

#### Possibilidades e melhorias a serem feitas

* evitar dados duplicados em instâncias de objeto Fila()
* melhor geração de senha para guichê
* fornecer posiçao do usuário na fila
* para cada especialidade, poderia haver um critério como "grau de complexidade"

fontes:
    https://www.camara.leg.br/noticias/630767-PESSOAS-MAIS-IDOSAS-PODERAO-TER-PRIORIDADE-NO-ATENDIMENTO